# Lokaverkefni: Leiðarval hjá Strætó

In [ ]:
from posixpath import split

def convertTime(t1, t2):
  t1_hours = int(t1.split(':')[0])
  t1_min = int(t1.split(':')[1])
  t1_seconds = int(t1.split(':')[2])

  t2_hours = int(t2.split(':')[0])
  t2_min = int(t2.split(':')[1])
  t2_seconds = int(t2.split(':')[2])


  t1_minutes = (t1_hours * 60 ) + t1_min + (t1_seconds / 60)
  t2_minutes = (t2_hours * 60 ) + t2_min + (t2_seconds / 60)

  diff = round((max(t1_minutes, t2_minutes) - min(t1_minutes, t2_minutes)), 2)

  return diff


t1 = '09:30:20'
t2 = '09:10:00'
print(convertTime(t1, t2))

In [ ]:
def time_to_minutes(ti):
  splitted = ti.split(':')
  
  hours = int(splitted[0])
  min = int(splitted[1])
  seconds = int(splitted[2])

  return (hours * 60) + min + (seconds / 60)

In [ ]:
# Graph stuff

def add_vertex(g, v):
  if v in g:
    raise Exception("Hnúturinn er þegar í netinu")
  else:
    g[v] = []
  return g

def add_edge(g, fro, to, data):
  if fro not in g:
    raise Exception(fro + " er ekki í netinu")
  if to not in g:
    raise Exception(to + " er ekki í netinu")
  g[fro].append([data, to])
  return g

def get_edges(g, v):
  return g[v]

In [ ]:
import csv
import sys

#0: stop_id
#1: stop_name
#2: stop_lat
#3: stop_lon
#4: location_type
stoppistodvar = []
with open(sys.path[0]+'/gtfs/stops.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) # skip the header
    for row in spamreader:
        stoppistodvar.append(row)

#0: route_id
#1: service_id
#2: trip_id
#3: trip_headsign
#4: trip_short_name
#5: direction_id
#6: block_id
#7: shape_id
trip_to_straeto = {}
with open(sys.path[0]+'/gtfs/trips.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) # skip the header
    for row in spamreader:
        if row[0][:2] == 'ST' and row[0] not in ['ST.101','ST.102','ST.103','ST.104','ST.105','ST.106']:
            trip_to_straeto[row[2]] = row[0]

#0: trip_id
#1: arrival_time
#2: departure_time
#3: stop_id
#4: stop_sequence
#5: stop_headsign
#6: pickup_type
leidir = []
with open(sys.path[0]+'/gtfs/stop_times.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) # skip the header
    paths = {}
    for row in spamreader:
      if row[6] == '0' and row[0] in trip_to_straeto:
        if row[0] not in paths:
          paths[row[0]] = []
        paths[row[0]].append(row)

stoppistodvar_id_to_name = {}
for x in stoppistodvar:
    stoppistodvar_id_to_name[x[0]] = x[1]

In [ ]:
# Making the graph

graph = {}
vertex_info = {}

for x in stoppistodvar:
  graph = add_vertex(graph, x[0])
  vertex_info[x[0]] = [x[0], stoppistodvar_id_to_name[x[0]], 'stöð']

for path in paths:
  for s in range(len(paths[path])-1):
    interval_name = 'p' + path + '_' + paths[path][s][4]
    graph = add_vertex(graph, interval_name)
    vertex_info[interval_name] = [path, trip_to_straeto[path], 'strætó']
    graph = add_edge(graph, paths[path][s][3], interval_name, paths[path][s][2])
    graph = add_edge(graph, interval_name, paths[path][s+1][3], paths[path][s+1][1])

# fjöldi hnúta
print(len(graph))
# fjöldi leggja
print(sum([len(graph[x]) for x in graph]))

In [ ]:
import math
import sys

stoppistodvar = []
with open(sys.path[0]+'/gtfs/stops.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) 
    for row in spamreader:
        stoppistodvar.append(row)

stop_locations = {}
for row in stoppistodvar:
    stop_id = row[0]
    stop_lat = float(row[2])
    stop_lon = float(row[3])
    stop_locations[stop_id] = (stop_lat, stop_lon)

lat, lon = stop_locations['10000802']
# print("Latitude:", lat, "Longitude:", lon)

def calculate_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    R = 6371000
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

# print('Distance in meters', calculate_distance(stop_locations['20000250'], stop_locations['20000251']))

MAX_WALKING_DISTANCE = 300  

def add_walking_edges(graph, stop_locations):
    stop_ids = list(stop_locations.keys())
    for i, stop_id1 in enumerate(stop_ids):
        for stop_id2 in stop_ids[i+1:]:  
            distance = calculate_distance(stop_locations[stop_id1], stop_locations[stop_id2])
            if distance <= MAX_WALKING_DISTANCE:
                try:
                    graph = add_edge(graph, stop_id1, stop_id2, distance)
                    graph = add_edge(graph, stop_id2, stop_id1, distance)
                except Exception as e:
                    print('Exception', e) 
    return graph

# add_walking_edges(graph, stop_locations)

def print_walkable_stops_from(graph, stop_id, stop_locations):
    if stop_id not in stop_locations:
        print("Stop ID not found in stop locations.")
        return
    
    origin = stop_locations[stop_id]
    print(f"Walkable locations from stop {stop_id}:")
    
    for other_stop_id, destination in stop_locations.items():
        if stop_id == other_stop_id:  
            continue
        
        distance = calculate_distance(origin, destination)
        if distance <= MAX_WALKING_DISTANCE:
            print(f"- To stop {other_stop_id}: {distance:.2f} meters")

print_walkable_stops_from(graph, '10000814', stop_locations)


In [ ]:
# Making the graph with walking edges

walk_graph = {}

for x in stoppistodvar:
  walk_graph = add_vertex(walk_graph, x[0])
walk_graph = add_walking_edges(walk_graph, stop_locations)

print(len(walk_graph))

for x in walk_graph['20000254']:
  print(x)

In [ ]:
def minutes_to_time(t):
    t = int(t)
    hours = t // 60
    minutes = t % 60
    return f'{hours:02d}:{minutes:02d}'

print(minutes_to_time(760))

In [ ]:
# priority queue

class PriorityQueue:
    heap = []
    pointer_list = {}
    def pop(self):
        print("pop", self.heap[:7])
        top = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.pointer_list[self.heap[0][0]] = 0
        self.pointer_list[top[0]] = len(self.heap)-1
        self.heap = self.heap[:-1]
        self.sift_down(0)
        return top
    def add(self, name, val):
        self.heap.append([name, val])
        self.pointer_list[name] = len(self.heap)-1
        self.sift_up(len(self.heap)-1)
    def update(self, name, val):
        if name not in self.pointer_list:
            self.add(name, val)
            return
        where = self.pointer_list[name]
        print("update", name, val, where, self.heap[where])
        self.heap[where][1] = val
        self.sift_up(where)
    def sift_up(self, index):
        if index == 0:
            return
        above = (index-1) // 2
        if self.heap[above][1] > self.heap[index][1]:
            temp = self.heap[above]
            self.heap[above] = self.heap[index]
            self.heap[index] = temp
            self.pointer_list[self.heap[above][0]] = above
            self.pointer_list[self.heap[index][0]] = index
            print("siftup", self.heap[above], self.heap[index])
            self.sift_up(above)
    def sift_down(self, index):
        if index * 2 + 1 >= len(self.heap):
            return
        below_1 = index*2
        below_2 = index*2 + 1
        if self.heap[below_1][1] < self.heap[below_2][1]:
            if self.heap[below_1][1] < self.heap[index][1]:
                temp = self.heap[below_1]
                self.heap[below_1] = self.heap[index]
                self.heap[index] = temp
                self.pointer_list[self.heap[below_1][0]] = below_1
                self.pointer_list[self.heap[index][0]] = index
                print("siftdown", self.heap[index], self.heap[below_1])
                self.sift_down(below_1)
        else:
            if self.heap[below_2][1] < self.heap[index][1]:
                temp = self.heap[below_2]
                self.heap[below_2] = self.heap[index]
                self.heap[index] = temp
                self.pointer_list[self.heap[below_2][0]] = below_2
                self.pointer_list[self.heap[index][0]] = index
                print("siftdown", self.heap[index], self.heap[below_2])
                self.sift_down(below_2)
        

In [ ]:
# g: graph
# g_walk: graph of walkable
# x: origin
# y: destination
# t: beginning time

def find_path(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")
    unvisited_nodes = list(g.keys())
    earliest = {}
    previous_nodes = {}
    for node in unvisited_nodes:
        earliest[node] = sys.maxsize
    earliest[x] = t
    while unvisited_nodes: # Need to find an algorithm that isn't O(n^2)
        # Change to use a priority queue instead of searching through the entire list of unvisited nodes?
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif earliest[node] < earliest[current_min_node]:
                current_min_node = node
        if earliest[current_min_node] == sys.maxsize:
            break
        e = get_edges(g, current_min_node)
        if current_min_node == y:
            break
        for edge in e:
            if earliest[edge[1]] >= time_to_minutes(edge[0]) and time_to_minutes(edge[0]) >= earliest[current_min_node]:
                earliest[edge[1]] = time_to_minutes(edge[0])
                previous_nodes[edge[1]] = current_min_node
        if current_min_node in g_walk:
            e = get_edges(g_walk, current_min_node)
            for edge in e:
                time_to_reach = earliest[current_min_node] + edge[0]/40 # 40 meters per minute
                if earliest[edge[1]] >= time_to_reach:
                    earliest[edge[1]] = time_to_reach
                    previous_nodes[edge[1]] = current_min_node
        unvisited_nodes.remove(current_min_node)
    output = []
    bus_trip = y
    output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes[bus_trip]
        output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return output[::-1]

meistaravellir_to_fb = find_path(graph, walk_graph, '90000097', '90000446', "11:30:00")
# meistaravellir_to_fb = find_path(graph, {}, '90000097', '90000446', "11:30:00")

In [ ]:
import time

def time_find_path(graph, walk_graph, test_cases):
    elapsed_times = [] 
    for x, y, t in test_cases:
        start_time = time.time()
        find_path(graph, walk_graph, x, y, t)
        end_time = time.time()
        elapsed = end_time - start_time  
        elapsed_times.append(elapsed)
    
    average_time = sum(elapsed_times) / len(elapsed_times)
    requests_per_second = 1 / average_time if average_time > 0 else float('inf')
    print(f"Average execution time: {average_time:.4f} seconds")
    print(f"Requests per second: {requests_per_second:.2f}")


test_cases = [
    ('90000097', '90000446', "11:30:00"), # meistaravellir to fb
    ('10000802', '10000804', "12:30:00"), # hamraborg to sunnuhlíð
    ('90000410', '90000296', "08:00:00"), # skógarás to háskóli íslands
    ('90000780', '10000875', "15:50:00"), # borgartún to smáralind
    ('14001641', '90000173', "17:30:00"), # flatahraun to dunhagi
    ('13001337', '13001300', "18:09:00"), # Asparlundur to Ásgarður
    ('16040620', '90000213', "20:30:00"), # Fmos to laugardalslaug
    ('90000351', '90000461', "11:43:00"), # ártún to kringlan
    ('90000077', '90000301', "09:15:00"), # bústaðakirkja to perlan
    ('90000446', '90000097', "11:30:00"), # fb to meistaravellir
    ]

time_find_path(graph, walk_graph, test_cases)


In [62]:
# Priority queue version (WORK IN PROGRESS)

# g: graph
# g_walk: graph of walkable
# x: origin
# y: destination
# t: beginning time

def find_path(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")
    unvisited_nodes = list(g.keys())
    earliest = {}
    previous_nodes = {}
    priority = PriorityQueue()
    for node in unvisited_nodes:
        earliest[node] = sys.maxsize
        #priority.add(node, sys.maxsize)
    earliest[x] = t
    priority.update(x, t)
    while unvisited_nodes: # Need to find an algorithm that isn't O(n^2)
        # Change to use a priority queue instead of searching through the entire list of unvisited nodes?
        current_min_node = priority.pop()
        if(current_min_node[1] != earliest[current_min_node[0]]):
            print(current_min_node, earliest[current_min_node[0]])
            raise Exception()
        current_min_node = current_min_node[0]
        if earliest[current_min_node] == sys.maxsize:
            print("!")
            break
        e = get_edges(g, current_min_node)
        if current_min_node == y:
            print("?")
            break
        for edge in e:
            if earliest[edge[1]] > time_to_minutes(edge[0]) and time_to_minutes(edge[0]) >= earliest[current_min_node]:
                print(edge[1], earliest[edge[1]], time_to_minutes(edge[0]))
                earliest[edge[1]] = time_to_minutes(edge[0])
                print("set_earliest", edge[1], time_to_minutes(edge[0]))
                priority.update(edge[1], time_to_minutes(edge[0]))
                previous_nodes[edge[1]] = current_min_node
        if current_min_node in g_walk:
            e = get_edges(g_walk, current_min_node)
            for edge in e:
                time_to_reach = earliest[current_min_node] + edge[0]/40 # 40 meters per minute
                if earliest[edge[1]] >= time_to_reach:
                    earliest[edge[1]] = time_to_reach
                    print("set_earliest_walk", edge[1], time_to_reach)
                    priority.update(edge[1], time_to_reach)
                    previous_nodes[edge[1]] = current_min_node
        unvisited_nodes.remove(current_min_node)
    output = []
    bus_trip = y
    output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    print(bus_trip, vertex_info[bus_trip], minutes_to_time(earliest[bus_trip]))
    while bus_trip != x:
        bus_trip = previous_nodes[bus_trip]
        print(bus_trip, vertex_info[bus_trip], minutes_to_time(earliest[bus_trip]))
        output.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return output[::-1]

meistaravellir_to_fb = find_path(graph, walk_graph, '90000097', '90000446', "11:30:00")
#meistaravellir_to_fb = find_path(graph, {}, '90000097', '90000446', "11:30:00")

update 90000097 690.0 0 ['90000097', 690.0]
pop [['90000097', 690.0]]
p526936_21 9223372036854775807 945.0
set_earliest p526936_21 945.0


IndexError: list index out of range

In [68]:
import heapq

def find_path_new(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")

    pq = []
    heapq.heappush(pq, (t, x))

    earliest = {node: float('inf') for node in g}
    earliest[x] = t

    previous_nodes = {}


    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_node == y:
            break

        if current_distance > earliest[current_node]:
            continue

        
        for edge in get_edges(g, current_node):
            next_stop, travel_time = edge[1], time_to_minutes(edge[0])
            new_distance = current_distance + travel_time
            if new_distance < earliest[next_stop]:
                earliest[next_stop] = new_distance
                previous_nodes[next_stop] = current_node
                heapq.heappush(pq, (new_distance, next_stop))

        if current_node in g_walk:
            for edge in get_edges(g_walk, current_node):
                next_stop, walking_time = edge[1], edge[0] / 40  
                new_distance = current_distance + walking_time
                if new_distance < earliest[next_stop]:
                    earliest[next_stop] = new_distance
                    previous_nodes[next_stop] = current_node
                    heapq.heappush(pq, (new_distance, next_stop))

    path = []
    bus_trip = y
    path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes.get(bus_trip)
        if bus_trip is None:
            return []  
        path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return path[::-1] # skila réttri röð

meistaravellir_to_fb = find_path_new(graph, walk_graph, '90000097', '90000446', "11:30:00")
print(meistaravellir_to_fb)


[['90000097', 'Meistaravellir', 'stöð', '11:30'], ['90000117', 'Grandavegur', 'stöð', '11:34'], ['90000168', 'Ásvallagata', 'stöð', '11:42'], ['90000180', 'Landakot', 'stöð', '11:46'], ['527046', 'ST.13', 'strætó', '18:28'], ['90000057', 'Ráðhúsið', 'stöð', '25:11'], ['90000055', 'Lækjartorg A', 'stöð', '25:19'], ['90000002', 'Þjóðleikhúsið', 'stöð', '25:25'], ['90000822', 'Bíó Paradís', 'stöð', '25:32'], ['90000821', 'Bíó Paradís', 'stöð', '25:34'], ['90000823', 'Barónsstígur', 'stöð', '25:40'], ['90020295', 'Hlemmur B', 'stöð', '25:47'], ['525486', 'ST.4', 'strætó', '32:20'], ['90000212', 'Höfðatorg', 'stöð', '38:54'], ['90000780', 'Borgartún', 'stöð', '39:01'], ['90000182', 'Sóltún', 'stöð', '39:08'], ['90000037', 'Tún', 'stöð', '39:15'], ['525486', 'ST.4', 'strætó', '45:52'], ['90000124', 'Lágmúli', 'stöð', '52:31'], ['90000125', 'Samgöngustofa', 'stöð', '52:38'], ['525486', 'ST.4', 'strætó', '59:18'], ['90000126', 'Háaleitisbraut', 'stöð', '65:59'], ['90000127', 'Fellsmúli', 'stöð

In [59]:
import heapq

def find_path_new(g, g_walk, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")

    pq = []
    heapq.heappush(pq, (t, x))

    earliest = {node: float('inf') for node in g}
    earliest[x] = t

    previous_nodes = {}


    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_node == y:
            break

        if current_distance > earliest[current_node]:
            continue

        
        for edge in get_edges(g, current_node):
            next_stop, travel_time = edge[1], time_to_minutes(edge[0])
            new_distance = current_distance + travel_time
            if new_distance < earliest[next_stop]:
                earliest[next_stop] = new_distance
                previous_nodes[next_stop] = current_node
                heapq.heappush(pq, (new_distance, next_stop))

        if current_node in g_walk:
            for edge in get_edges(g_walk, current_node):
                next_stop, walking_time = edge[1], edge[0] / 40  
                new_distance = current_distance + walking_time
                if new_distance < earliest[next_stop]:
                    earliest[next_stop] = new_distance
                    previous_nodes[next_stop] = current_node
                    heapq.heappush(pq, (new_distance, next_stop))

    path = []
    bus_trip = y
    path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    while bus_trip != x:
        bus_trip = previous_nodes.get(bus_trip)
        if bus_trip is None:
            return []  
        path.append(vertex_info[bus_trip] + [minutes_to_time(earliest[bus_trip])])
    return path[::-1] # skila réttri röð

meistaravellir_to_fb = find_path_new(graph, walk_graph, '90000097', '90000446', "11:30:00")
print(meistaravellir_to_fb)


Average execution time: 0.1654 seconds
Requests per second: 6.05


In [ ]:
import time

def time_find_path_new(graph, walk_graph, test_cases):
    elapsed_times = [] 
    for x, y, t in test_cases:
        start_time = time.time()
        find_path_new(graph, walk_graph, x, y, t)
        end_time = time.time()
        elapsed = end_time - start_time 
        elapsed_times.append(elapsed)
    
    average_time = sum(elapsed_times) / len(elapsed_times)
    requests_per_second = 1 / average_time if average_time > 0 else float('inf')
    print(f"Average execution time: {average_time:.4f} seconds")
    print(f"Requests per second: {requests_per_second:.2f}")


test_cases = [
    ('90000097', '90000446', "11:30:00"), # meistaravellir to fb
    ('10000802', '10000804', "12:30:00"), # hamraborg to sunnuhlíð
    ('90000410', '90000296', "08:00:00"), # skógarás to háskóli íslands
    ('90000780', '10000875', "15:50:00"), # borgartún to smáralind
    ('14001641', '90000173', "17:30:00"), # flatahraun to dunhagi
    ('13001337', '13001300', "18:09:00"), # Asparlundur to Ásgarður
    ('16040620', '90000213', "20:30:00"), # Fmos to laugardalslaug
    ('90000351', '90000461', "11:43:00"), # ártún to kringlan
    ('90000077', '90000301', "09:15:00"), # bústaðakirkja to perlan
    ('90000446', '90000097', "11:30:00"), # fb to meistaravellir
    ]

time_find_path_new(graph, walk_graph, test_cases)
